In [49]:
import os

os.chdir('../')

In [50]:
pwd

"E:\\Father's_Document\\code\\MlOps_project"

In [51]:
os.chdir(f"E:\\Father's_Document\\code\\MlOps_project\\End_to_End_wine_price_prediction")

In [52]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Jha01Sarvjeet/End_to_End_wine_quality_prediction.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Jha01Sarvjeet'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd746f73e322061c0553fb781ba70e90f616aa10d'

In [53]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [54]:
from src.wine_qulity_predction.utils.common import read_yaml, create_directories, save_json
from src.wine_qulity_predction.constants import *

In [55]:
class ModelConfiguratoinManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH, ):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(root_dir=config.root_dir,
                                                        test_data_path=config.test_data_path,
                                                        model_path=config.model_path,
                                                        metric_file_name=config.metric_file_name,
                                                        all_params=params,
                                                        mlflow_uri='https://dagshub.com/Jha01Sarvjeet/End_to_End_wine_quality_prediction.mlflow',
                                                        target_column=schema.name,
                                                        )

        return model_evaluation_config








In [56]:
import numpy as np
import pandas as pd
import mlflow
import joblib
from urllib.parse import urlparse
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_squared_error, r2_score

In [57]:
class MedelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = root_mean_squared_error(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self, ):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_parse = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            #saving metrics as local
            scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            #model registry does not work with file store
            if tracking_url_type_parse != 'file':
                mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path="model",
                # Remove any newer parameters that might cause issues
            )



            else:
                mlflow.sklearn.log_model(sk_model=model, artifact_path="model")









In [59]:
pwd

"E:\\Father's_Document\\code\\MlOps_project\\End_to_End_wine_price_prediction"

In [60]:
try:
    config = ModelConfiguratoinManager()
    model_evaluation_config = config.get_model_evaluation_config()

    model_evaluation = MedelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-01 12:51:05,401 : INFO : common : yaml file : config\config.yml loadded successfully] 
[2025-07-01 12:51:05,401 : INFO : common : yaml file : schema.yml loadded successfully] 
[2025-07-01 12:51:05,401 : INFO : common : yaml file : params.yml loadded successfully] 
[2025-07-01 12:51:05,401 : INFO : common : Created directory at artifacts] 
[2025-07-01 12:51:05,401 : INFO : common : Created directory at artifacts/model_evaluation] 
[2025-07-01 12:51:06,206 : INFO : common : json file saved at artifacts\model_evaluation\metrics.json] 
